In [1]:
import sys
sys.path.append("../../classes/")
import logging
import getpass 
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from iac import iac


# Provide IAM credentials

In [2]:
KEY = getpass.getpass('Enter user name: ')
SECRET = getpass.getpass('Enter password')



Enter user name: ········
Enter password········


In [3]:
IAC = iac(KEY,SECRET,region='us-west-2')

# S3 buckets

In [4]:
IAC.S3_list()

{'us-west-2': ['dend-lake']}

In [5]:
IAC.S3_create('dend-lake-test-to-delete')

True

In [6]:
IAC.S3_list()

{'us-west-2': ['dend-lake', 'dend-lake-test-to-delete']}

In [7]:
IAC.S3_delete('dend-lake-test-to-delete')

True

In [8]:
IAC.S3_list()

{'us-west-2': ['dend-lake']}

# Redshift user role
For a Redshift cluster and our project, we need an S3 read-only access from within a Redshift cluster. Using my predefined functions:

# Redshift cluster

In [5]:
IAC.redshift_create("dwh.cfg")

Loaded config file...
Creating a redshift cluster... Run 'redshift_properties(config_file)' and store the Redshift endpoint address in config file in section ['DWH'][''DWH_ENDPOINT']!  


In [9]:
props = IAC.redshift_properties("dwh.cfg")
props

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,admin
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.cgxtkssnactv.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-7d049f05
7,NumberOfNodes,4


In [10]:
VPC_ID = (props.loc[props['Key'] == 'VpcId'])['Value'].tolist()[0]


In [11]:
conn_string = IAC.redshift_authorize_connection("dwh.cfg",VPC_ID)

ec2.SecurityGroup(id='sg-018e9b9dc477c1887')
Could not authorize ingress or ingress already present. Read the error below:
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


In [12]:
%load_ext sql
%sql $conn_string

'Connected: admin@dwh'

In [8]:
IAC.redshift_delete("dwh.cfg")

Loaded config file...
Deleting cluster


{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'admin',
  'DBName': 'dwh',
  'Endpoint': {'Address': 'dwhcluster.cgxtkssnactv.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2020, 4, 16, 13, 31, 56, 59000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-927183c8',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-7d049f05',
  'AvailabilityZone': 'us-west-2c',
  'PreferredMaintenanceWindow': 'sat:08:30-sat:09:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': 

In [9]:

IAC.redshift_properties("dwh.cfg")

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,deleting
3,MasterUsername,admin
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.cgxtkssnactv.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-7d049f05
7,NumberOfNodes,4
